In [49]:
from __future__ import division
from IPython.display import display
import scipy.stats as sts
import numpy as np
import math as m
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import ks_2samp

In [50]:
class Empirica:
    def Fn(X,a):
        v = len([x for x in X if x<=a])/len(X)
        return v
    def bandadeconfianza(X,a):
        y = np.arange(min(X),max(X)+0.02,0.01)
        n = len(X)
        En = m.sqrt(m.log(2/a)/(2*n))
        L =[max(Fn(x,X)-En,0) for x in np.arange(min(X),max(X)+0.02,0.01)]
        U =[min(Fn(x,X)+En,1) for x in np.arange(min(X),max(X)+0.02,0.01)]
        F = [Fn(x,X) for x in y ]
        return plt.plot(y,U,y,L,y,F)
    def Fn2(X,Y):
        test_stat = ks_2samp(X,Y)
        return test_stat

In [348]:
class Estimadores:
    def mu(X):
        z = np.mean(X)
        return z
    def sigma(X):
        w = np.std(X)
        return w
    def alpha(X):
        u = 1/np.mean(X)
        return u
    def a(X):
        s = min(X)
        return s
    def b(X):
        m = max(X)
        return m
    def c_gamma(X):
        i = ((np.mean(X))**2)/((np.std(X))**2)
        return i
    def d_gamma(X):
        i = (np.mean(X)/((np.std(X))**2))
        return i
    def c_beta(X):
        i =  ((np.mean(X))*((1/np.std(X))**2))*((np.mean(X))*(1-(np.mean(X)))-((np.std(X))**2))
        return i
    def d_beta(X):
        i = (((1/np.std(X))**2)*(((np.mean(X))*((1-np.mean(X))**2))))-((1-(np.mean(X))))
        return i

In [349]:
F = {'Normal':lambda x:sts.norm.cdf(x,Estimadores.mu(X),Estimadores.sigma(X)),
        'Exponencial':lambda x:sts.expon.cdf(x,Estimadores.alpha(X)),
        'Uniforme':lambda x:sts.uniform.cdf(x,Estimadores.a(X),Estimadores.b(X)),
        'Beta':lambda x:sts.beta.cdf(x,Estimadores.c_beta(X),Estimadores.d_beta(X)),
        'Gamma':lambda x:sts.gamma.cdf(x,Estimadores.c_gamma(X),Estimadores.d_gamma(X))}

In [350]:
class PruebasBDA:
    def KS(X):
        n = len(X)
        Xi = sorted(X)
        Dn = []
        for k in F:
            Dn.append(max([(max((i+1)/n-F[k](Xi[i]),F[k](Xi[i])-(i/n))) for i in range(n)]))
        return Dn
    def Cn(X):
        n = len(X)
        Xi = sorted(X)
        Dn = []
        for k in F:
            Dn.append((12*n*(sum([((F[k](Xi[i])-(2*(i+1)-1)/n)**2) for i in range(n)]))+1)/(12*n))
        return Dn

In [351]:
class ValoresBDA:
    def nDn(X):
        n = len(X)
        Xi = sorted(X)
        Dn = []
        for k in F:
            Dn.append(m.sqrt(n)*(max([(max((i+1)/n-F[k](Xi[i]),F[k](Xi[i])-(i/n))) for i in range(n)])))
        return Dn

In [364]:
class Resultados:
    def Graficas(X,alpha):
        display(pd.DataFrame([Estimadores.mu(X),Estimadores.sigma(X),Estimadores.alpha(X),Estimadores.a(X),Estimadores.b(X),Estimadores.c_beta(X),Estimadores.d_beta(X),Estimadores.c_gamma(X),Estimadores.d_gamma(X)],index=['$\mu$','$\sigma^2$','$\\alpha$','$\min_a$','$\max_b$','$\\alpha_{beta}$','$\\beta_{beta}$','$\\alpha_{gamma}$','$\\beta_{gamma}$']).T)
        nDn =lambda l=1:1-2*sum(((-1)**(j-1))*m.exp(-2*(j**2)*(l**2)) for j in range(1,300))
        Quant = lambda alpha:max(q for q in np.arange(0.01,4,0.01) if nDn(q)<alpha)
        w = len(PruebasBDA.KS(X))
        ñ = len(ValoresBDA.nDn(X))
        Comparacion1 = pd.DataFrame({
        '$\ Dist$': ['Normal','Exponencial','Uniforme','Beta','Gamma'],
        '$\ D_n$': [PruebasBDA.KS(X)[i] for i in range (w)],
        '$\sqrt{n}D_n$': [ValoresBDA.nDn(X)[i] for i in range(ñ)],
        '$\ Q_{%s}$'%alpha:[Quant(alpha),Quant(alpha),Quant(alpha),Quant(alpha),Quant(alpha)],
        '$\ H_0$':[Quant(alpha)>=ValoresBDA.nDn(X)[i] for i in range(ñ)]
    }, columns=['$\ Dist$','$\ D_n$','$\sqrt{n}D_n$','$\ Q_{%s}$'%alpha,'$\ H_0$'])
        display(pd.DataFrame([PruebasBDA.Cn(X)[i] for i in range(w)],index=['$\ Normal$','$\ Exponencial$','$\ Uniforme$','$\ Beta$','$\ Gamma$']).T)
        Fn = lambda y:len([x for x in X if x<=y])/len(X)
        t = np.arange(min(X),max(X)+0.02,0.01)
        Femp = [Fn(x) for x in t ]
        plt.plot(t,Femp,color="black", label ="Empírica")
        [plt.plot(t,F[x](t),label=x) for x in F.keys()]
        plt.legend(loc="lower right")
        return Comparacion1

In [ ]:
###################################################################################

In [161]:
class InferenciaNoParametrica:
    def Wilcoxon(X,Y,a):
        n = int(len(X))
        m = int(len(Y))
        print("n= {}, m= {} ".format(n,m))
        X = [(x,"X") for x in X]
        Y = [(y,"Y") for y in Y]
        Z = sorted(X+Y)
        S = [(i+1)*(Z[i][1]=="Y") for i in range(len(Z))]
        W = sum(S)
        E_W = (n*(m+n+1))/2
        V_W = (n*m*(n+m+1))/12
        W_ = (W-E_W)/((V_W)**(1/2))
        sup = -np.percentile(np.random.standard_normal(100000),a*100/2)
        inf = np.percentile(np.random.standard_normal(100000),a*100/2)
        if inf <= W_ and W_ <= sup:
            print ("No rechazar\nW={} esta en el intervalo ({},{})".format(W_,inf,sup))
        else:
            print ("Rechazar\nW={} no esta en el intervalo ({},{})".format(W_,inf,sup))
    def WhitneyW(X,Y,a):
        n = int(len(X))
        m = int(len(Y))
        print("n= {}, m= {} ".format(n,m))
        Dn = []
        for j in range(m):
            Dn.append(sum([X[i]<Y[j] for i in range(n)]))        
        W = sum(Dn)
        E_W = (n*(m+n+1)+n*(n+1))/2
        V_W = (n*m*(n+m+1))/12
        W_ = (W-E_W)/((V_W)**(1/2))
        sup = -np.percentile(np.random.standard_normal(100000),a*100/2)
        inf = np.percentile(np.random.standard_normal(100000),a*100/2)
        if inf <= W_ and W_ <= sup:
            print ("No rechazar\nWW={} esta en el intervalo ({},{})".format(W_,inf,sup))
        else:
            print ("Rechazar\nWW={} no esta en el intervalo ({},{})".format(W_,inf,sup))
    def Fisher(X,Y,a):
        n = int(len(X))
        m = int(len(Y))
        print("n= {}, m= {} ".format(n,m))
        Dn = []
        for i in range(n):
            Dn.append(abs(X[i]-Y[i]))
            o = sorted(Dn)
        Zn = []
        for j in range(n):
            Zn.append(int(Y[j]>X[j])*(j+1))
        W = sum(Zn)
        E_W = (n*(n+1))/4
        V_W = (n*(m+1)*(n+m+1))/24
        W_ = (W-E_W)/((V_W)**(1/2))
        sup = -np.percentile(np.random.standard_normal(100000),a*100/2)
        inf = np.percentile(np.random.standard_normal(100000),a*100/2)
        if inf <= W_ and W_ <= sup:
            print ("No rechazar\nF={} esta en el intervalo ({},{})".format(W_,inf,sup))
        else:
            print ("Rechazar\nF={} no esta en el intervalo ({},{})".format(W_,inf,sup))
    def PruebaKendall(X,Y,a):
        Z = list(zip(X,Y))
        m = len(Y)
        n = len(Z)
        print("n= {}, m= {} ".format(n,m))
        k = 0
        for i in range(n-1):
            for j in range(i+1,n):
                k+= (1*((Z[i][0]-Z[j][0])*(Z[i][1]-Z[j][1])>0)+(-1*((Z[i][0]-Z[j][0])*(Z[i][1]-Z[j][1])<0)))
        E_W = 0
        V_W = (n*(n-1)*(2*n+5))/18
        W_ = (k-E_W)/((V_W)**(1/2))
        sup = -np.percentile(np.random.standard_normal(100000),a*100/2)
        inf = np.percentile(np.random.standard_normal(100000),a*100/2)
        print("k={}".format(k))
        if inf <= W_ and W_ <= sup:
            print ("No rechazar\nPK={} esta en el intervalo ({},{})".format(W_,inf,sup))
        else:
            print ("Rechazar\nPK={} no esta en el intervalo ({},{})".format(W_,inf,sup))

In [ ]:
class Remuestreo:
    def Jacknife(X,a,b):
        